In [1]:
# Install the necessary libraries
#!pip install transformers torch

# Import the required libraries
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the T5 model and tokenizer
model_name = "mrm8488/t5-base-finetuned-wikiSQL"
tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Define the prompts based on your dataset
prompts = [
    "Get all documents",
    "Get documents from index users"
]

# Function to generate SQL queries from prompts
def generate_sql(prompt):
    # Tokenize the prompt
    inputs = tokenizer.encode("translate English to OpenSearch Query: " + prompt, return_tensors="pt", max_length=512, truncation=True)
    
    # Generate SQL query
    outputs = model.generate(inputs, max_length=150, num_beams=5, early_stopping=True)
    
    # Decode the generated text
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql_query

# Generate and print SQL queries for each prompt
for prompt in prompts:
    sql_query = generate_sql(prompt)
    print(f"Prompt: {prompt}")
    print(f"Generated SQL: {sql_query}")
    print()


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Prompt: Get all documents
Generated SQL: SELECT Documents FROM table WHERE Name = all

Prompt: Get documents from index users
Generated SQL: SELECT Documents from index users FROM table



In [3]:
# Let's finetune this model with SQL further with text_to_sql_dataset.csv

from datasets import load_dataset

# Define a preprocessing function
def preprocess_function(examples):
    inputs = examples['text']
    targets = examples['OpenSearchQuery']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Load dataset
dataset = load_dataset('csv', data_files={'train': 'opensearch_finetuning_dataset.csv'}, split='train')

# Tokenizer
from transformers import AutoTokenizer
model_name = "mrm8488/t5-base-finetuned-wikiSQL"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Preprocess the dataset
encoded_dataset = dataset.map(preprocess_function, batched=True)



The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Map:   0%|          | 0/94 [00:00<?, ? examples/s]

C:\Users\91998\PycharmProjects\pythonProject\venv\lib\site-packages\transformers\tokenization_utils_base.py:4144: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [4]:
from transformers import TrainingArguments

# Define training arguments with output_dir
training_args = TrainingArguments(
    output_dir='./results',  # Directory to save model checkpoints and logs
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
)


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments

# Load dataset
dataset = load_dataset('csv', data_files={'train': 'opensearch_finetuning_dataset.csv'}, split='train')

# Tokenizer
model_name = "mrm8488/t5-base-finetuned-wikiSQL"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a preprocessing function
def preprocess_function(examples):
    inputs = examples['text']
    targets = examples['OpenSearchQuery']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Preprocess the dataset
encoded_dataset = dataset.map(preprocess_function, batched=True)

# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define training arguments with output_dir
training_args = TrainingArguments(
    output_dir='./results',  # Directory to save model checkpoints and logs
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("./t5-base-finetuned-wikiSQL-os")
tokenizer.save_pretrained("./t5-base-finetuned-wikiSQL-os")

print("Model and tokenizer saved successfully.")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Step,Training Loss
10,11.111800
20,1.915500
30,0.562500
40,0.311900
50,0.241800
60,0.194500
70,0.171100
80,0.145200
90,0.148300


In [24]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the T5 model and tokenizer
model_name = "mrm8488/t5-base-finetuned-wikiSQL"
tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Define the prompts based on your dataset
prompts = [
    "Tell me names and ages of all employees",
    "Get the total sales amount of sales"
]

# Function to generate SQL queries from prompts
def generate_sql(prompt):
    # Tokenize the prompt
    inputs = tokenizer.encode("translate English to OpenSearch Query: " + prompt, return_tensors="pt", max_length=512, truncation=True)
    
    # Generate SQL query
    outputs = model.generate(inputs, max_length=150, num_beams=5, early_stopping=True)
    
    # Decode the generated text
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql_query

# Generate and print SQL queries for each prompt
for prompt in prompts:
    sql_query = generate_sql(prompt)
    print(f"Prompt: {prompt}")
    print(f"Generated SQL: {sql_query}")
    print()

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Prompt: Tell me names and ages of all employees
Generated SQL: SELECT Names and Ages FROM table WHERE Position = all employees

Prompt: Get the total sales amount of sales
Generated SQL: SELECT COUNT Sales FROM table WHERE Sales = sales



In [25]:
# Let's build the OpenSearch query dataset


import pandas as pd

# Define a more extensive dataset with 100 samples
data = {
    "text": [
        "translate English to OpenSearch query: Get all documents",
        "translate English to OpenSearch query: Get documents from index logs",
        "translate English to OpenSearch query: Count documents in index logs",
        "translate English to OpenSearch query: Search for error messages",
        "translate English to OpenSearch query: Search for user login events",
        "translate English to OpenSearch query: Aggregate logs by status",
        "translate English to OpenSearch query: Get documents from index users",
        "translate English to OpenSearch query: Find all documents with status failed",
        "translate English to OpenSearch query: Count documents with severity high",
        "translate English to OpenSearch query: Aggregate by user and count",
        "translate English to OpenSearch query: Get the document with id 1",
        "translate English to OpenSearch query: Delete document with id 1 from index users",
        "translate English to OpenSearch query: Update document with id 1 in index users",
        "translate English to OpenSearch query: Get all indexes",
        "translate English to OpenSearch query: Create a new index called test",
        "translate English to OpenSearch query: Delete index test",
        "translate English to OpenSearch query: Refresh index users",
        "translate English to OpenSearch query: Get settings for index users",
        "translate English to OpenSearch query: Update settings for index users",
        "translate English to OpenSearch query: Get mappings for index users",
        "translate English to OpenSearch query: Update mappings for index users",
        "translate English to OpenSearch query: Get aliases for index users",
        "translate English to OpenSearch query: Add alias test_alias to index users",
        "translate English to OpenSearch query: Remove alias test_alias from index users",
        "translate English to OpenSearch query: Get documents with name John",
        "translate English to OpenSearch query: Get documents with age greater than 30",
        "translate English to OpenSearch query: Get documents with age less than 30",
        "translate English to OpenSearch query: Get documents with age between 20 and 30",
        "translate English to OpenSearch query: Get documents with status active",
        "translate English to OpenSearch query: Get documents with status inactive",
        "translate English to OpenSearch query: Get documents with date after 2020-01-01",
        "translate English to OpenSearch query: Get documents with date before 2020-01-01",
        "translate English to OpenSearch query: Aggregate by age",
        "translate English to OpenSearch query: Aggregate by age and count",
        "translate English to OpenSearch query: Aggregate by age and average",
        "translate English to OpenSearch query: Aggregate by age and sum",
        "translate English to OpenSearch query: Aggregate by status and count",
        "translate English to OpenSearch query: Aggregate by status and average",
        "translate English to OpenSearch query: Aggregate by status and sum",
        "translate English to OpenSearch query: Aggregate by status and min",
        "translate English to OpenSearch query: Aggregate by status and max",
        "translate English to OpenSearch query: Get documents with name starting with J",
        "translate English to OpenSearch query: Get documents with name ending with n",
        "translate English to OpenSearch query: Get documents with name containing ohn",
        "translate English to OpenSearch query: Get documents with status failed and date after 2021-01-01",
        "translate English to OpenSearch query: Get documents with severity high and date before 2021-01-01",
        "translate English to OpenSearch query: Aggregate by severity and count",
        "translate English to OpenSearch query: Aggregate by severity and average",
        "translate English to OpenSearch query: Aggregate by severity and sum",
        "translate English to OpenSearch query: Aggregate by severity and min",
        "translate English to OpenSearch query: Aggregate by severity and max",
        "translate English to OpenSearch query: Get documents with status failed and severity high",
        "translate English to OpenSearch query: Get documents with name John and age greater than 30",
        "translate English to OpenSearch query: Get documents with name John and age less than 30",
        "translate English to OpenSearch query: Get documents with name John and age between 20 and 30",
        "translate English to OpenSearch query: Get documents with name John and status active",
        "translate English to OpenSearch query: Get documents with name John and status inactive",
        "translate English to OpenSearch query: Get documents with name John and date after 2020-01-01",
        "translate English to OpenSearch query: Get documents with name John and date before 2020-01-01",
        "translate English to OpenSearch query: Get documents with status active and age greater than 30",
        "translate English to OpenSearch query: Get documents with status inactive and age less than 30",
        "translate English to OpenSearch query: Get documents with status active and date after 2020-01-01",
        "translate English to OpenSearch query: Get documents with status inactive and date before 2020-01-01",
        "translate English to OpenSearch query: Aggregate by status and severity",
        "translate English to OpenSearch query: Aggregate by status and age",
        "translate English to OpenSearch query: Aggregate by status and date",
        "translate English to OpenSearch query: Aggregate by severity and age",
        "translate English to OpenSearch query: Aggregate by severity and date",
        "translate English to OpenSearch query: Aggregate by age and date",
        "translate English to OpenSearch query: Aggregate by age and status",
        "translate English to OpenSearch query: Get documents with name John and severity high",
        "translate English to OpenSearch query: Get documents with name John and date after 2021-01-01",
        "translate English to OpenSearch query: Get documents with name John and date before 2021-01-01",
        "translate English to OpenSearch query: Get documents with status failed and age greater than 30",
        "translate English to OpenSearch query: Get documents with severity high and age less than 30",
        "translate English to OpenSearch query: Get documents with status failed and date after 2021-01-01",
        "translate English to OpenSearch query: Get documents with severity high and date before 2021-01-01",
        "translate English to OpenSearch query: Aggregate by status, severity and age",
        "translate English to OpenSearch query: Aggregate by status, severity and date",
        "translate English to OpenSearch query: Aggregate by severity, age and date",
        "translate English to OpenSearch query: Aggregate by age, status and date",
        "translate English to OpenSearch query: Get documents with name starting with A",
        "translate English to OpenSearch query: Get documents with name ending with Z",
        "translate English to OpenSearch query: Get documents with name containing BCD",
        "translate English to OpenSearch query: Get documents with status pending",
        "translate English to OpenSearch query: Get documents with severity medium",
        "translate English to OpenSearch query: Get documents with priority high",
        "translate English to OpenSearch query: Get documents with priority low",
        "translate English to OpenSearch query: Get documents with priority medium",
        "translate English to OpenSearch query: Aggregate by priority and count",
        "translate English to OpenSearch query: Aggregate by priority and average",
        "translate English to OpenSearch query: Aggregate by priority and sum",
        "translate English to OpenSearch query: Aggregate by priority and min",
        "translate English to OpenSearch query: Aggregate by priority and max"
    ],
    "OpenSearchQuery": [
        "GET /_search",
        "GET /logs/_search",
        "GET /logs/_count",
        "GET /_search { \"query\": { \"match\": { \"message\": \"error\" }}}",
        "GET /_search { \"query\": { \"match\": { \"event\": \"login\" }}}",
        "GET /_search { \"aggs\": { \"status\": { \"terms\": { \"field\": \"status\" }}}}",
        "GET /users/_search",
        "GET /_search { \"query\": { \"match\": { \"status\": \"failed\" }}}",
        "GET /_search { \"query\": { \"match\": { \"severity\": \"high\" }}}",
        "GET /_search { \"aggs\": { \"user\": { \"terms\": { \"field\": \"user\" }}}}",
        "GET /_search { \"query\": { \"match\": { \"_id\": \"1\" }}}",
        "DELETE /users/_doc/1",
        "POST /users/_update/1 { \"doc\": { \"field\": \"value\" }}",
        "GET /_cat/indices",
        "PUT /test",
        "DELETE /test",
        "POST /users/_refresh",
        "GET /users/_settings",
        "PUT /users/_settings { \"index\": { \"number_of_replicas\": 2 }}",
        "GET /users/_mapping",
        "PUT /users/_mapping { \"properties\": { \"field\": { \"type\": \"text\" }}}",
        "GET /users/_alias",
        "POST /users/_alias/test_alias",
        "DELETE /users/_alias/test_alias",
        "GET /_search { \"query\": { \"match\": { \"name\": \"John\" }}}",
        "GET /_search { \"query\": { \"range\": { \"age\": { \"gt\": 30 }}}}",
        "GET /_search { \"query\": { \"range\": { \"age\": { \"lt\": 30 }}}}",
        "GET /_search { \"query\": { \"range\": { \"age\": { \"gte\": 20, \"lte\": 30 }}}}",
        "GET /_search { \"query\": { \"match\": { \"status\": \"active\" }}}",
        "GET /_search { \"query\": { \"match\": { \"status\": \"inactive\" }}}",
        "GET /_search { \"query\": { \"range\": { \"date\": { \"gt\": \"2020-01-01\" }}}}",
        "GET /_search { \"query\": { \"range\": { \"date\": { \"lt\": \"2020-01-01\" }}}}",
        "GET /_search { \"aggs\": { \"age\": { \"terms\": { \"field\": \"age\" }}}}",
        "GET /_search { \"aggs\": { \"age\": { \"terms\": { \"field\": \"age\" }}, \"aggs\": { \"count\": { \"value_count\": { \"field\": \"_index\" }}}}}",
        "GET /_search { \"aggs\": { \"age\": { \"terms\": { \"field\": \"age\" }}, \"aggs\": { \"average\": { \"avg\": { \"field\": \"age\" }}}}}",
        "GET /_search { \"aggs\": { \"age\": { \"terms\": { \"field\": \"age\" }}, \"aggs\": { \"sum\": { \"sum\": { \"field\": \"age\" }}}}}",
        "GET /_search { \"aggs\": { \"status\": { \"terms\": { \"field\": \"status\" }}, \"aggs\": { \"count\": { \"value_count\": { \"field\": \"_index\" }}}}}",
        "GET /_search { \"aggs\": { \"status\": { \"terms\": { \"field\": \"status\" }}, \"aggs\": { \"average\": { \"avg\": { \"field\": \"status\" }}}}}",
        "GET /_search { \"aggs\": { \"status\": { \"terms\": { \"field\": \"status\" }}, \"aggs\": { \"sum\": { \"sum\": { \"field\": \"status\" }}}}}",
        "GET /_search { \"aggs\": { \"status\": { \"terms\": { \"field\": \"status\" }}, \"aggs\": { \"min\": { \"min\": { \"field\": \"status\" }}}}}",
        "GET /_search { \"aggs\": { \"status\": { \"terms\": { \"field\": \"status\" }}, \"aggs\": { \"max\": { \"max\": { \"field\": \"status\" }}}}}",
        "GET /_search { \"query\": { \"wildcard\": { \"name\": \"J*\" }}}",
        "GET /_search { \"query\": { \"wildcard\": { \"name\": \"*n\" }}}",
        "GET /_search { \"query\": { \"wildcard\": { \"name\": \"*ohn*\" }}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"status\": \"failed\" }}, { \"range\": { \"date\": { \"gt\": \"2021-01-01\" }}}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"severity\": \"high\" }}, { \"range\": { \"date\": { \"lt\": \"2021-01-01\" }}}]}}}",
        "GET /_search { \"aggs\": { \"severity\": { \"terms\": { \"field\": \"severity\" }}, \"aggs\": { \"count\": { \"value_count\": { \"field\": \"_index\" }}}}}",
        "GET /_search { \"aggs\": { \"severity\": { \"terms\": { \"field\": \"severity\" }}, \"aggs\": { \"average\": { \"avg\": { \"field\": \"severity\" }}}}}",
        "GET /_search { \"aggs\": { \"severity\": { \"terms\": { \"field\": \"severity\" }}, \"aggs\": { \"sum\": { \"sum\": { \"field\": \"severity\" }}}}}",
        "GET /_search { \"aggs\": { \"severity\": { \"terms\": { \"field\": \"severity\" }}, \"aggs\": { \"min\": { \"min\": { \"field\": \"severity\" }}}}}",
        "GET /_search { \"aggs\": { \"severity\": { \"terms\": { \"field\": \"severity\" }}, \"aggs\": { \"max\": { \"max\": { \"field\": \"severity\" }}}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"status\": \"failed\" }}, { \"match\": { \"severity\": \"high\" }}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"name\": \"John\" }}, { \"range\": { \"age\": { \"gt\": 30 }}}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"name\": \"John\" }}, { \"range\": { \"age\": { \"lt\": 30 }}}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"name\": \"John\" }}, { \"range\": { \"age\": { \"gte\": 20, \"lte\": 30 }}}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"name\": \"John\" }}, { \"match\": { \"status\": \"active\" }}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"name\": \"John\" }}, { \"match\": { \"status\": \"inactive\" }}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"name\": \"John\" }}, { \"range\": { \"date\": { \"gt\": \"2020-01-01\" }}}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"name\": \"John\" }}, { \"range\": { \"date\": { \"lt\": \"2020-01-01\" }}}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"status\": \"active\" }}, { \"range\": { \"age\": { \"gt\": 30 }}}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"status\": \"inactive\" }}, { \"range\": { \"age\": { \"lt\": 30 }}}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"status\": \"active\" }}, { \"range\": { \"date\": { \"gt\": \"2020-01-01\" }}}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"status\": \"inactive\" }}, { \"range\": { \"date\": { \"lt\": \"2020-01-01\" }}}]}}}",
        "GET /_search { \"aggs\": { \"status\": { \"terms\": { \"field\": \"status\" }}, \"aggs\": { \"severity\": { \"terms\": { \"field\": \"severity\" }}}}}",
        "GET /_search { \"aggs\": { \"status\": { \"terms\": { \"field\": \"status\" }}, \"aggs\": { \"age\": { \"terms\": { \"field\": \"age\" }}}}}",
        "GET /_search { \"aggs\": { \"status\": { \"terms\": { \"field\": \"status\" }}, \"aggs\": { \"date\": { \"terms\": { \"field\": \"date\" }}}}}",
        "GET /_search { \"aggs\": { \"severity\": { \"terms\": { \"field\": \"severity\" }}, \"aggs\": { \"age\": { \"terms\": { \"field\": \"age\" }}}}}",
        "GET /_search { \"aggs\": { \"severity\": { \"terms\": { \"field\": \"severity\" }}, \"aggs\": { \"date\": { \"terms\": { \"field\": \"date\" }}}}}",
        "GET /_search { \"aggs\": { \"age\": { \"terms\": { \"field\": \"age\" }}, \"aggs\": { \"date\": { \"terms\": { \"field\": \"date\" }}}}}",
        "GET /_search { \"aggs\": { \"age\": { \"terms\": { \"field\": \"age\" }}, \"aggs\": { \"status\": { \"terms\": { \"field\": \"status\" }}}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"name\": \"John\" }}, { \"match\": { \"severity\": \"high\" }}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"name\": \"John\" }}, { \"range\": { \"date\": { \"gt\": \"2021-01-01\" }}}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"name\": \"John\" }}, { \"range\": { \"date\": { \"lt\": \"2021-01-01\" }}}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"status\": \"failed\" }}, { \"range\": { \"age\": { \"gt\": 30 }}}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"severity\": \"high\" }}, { \"range\": { \"age\": { \"lt\": 30 }}}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"status\": \"failed\" }}, { \"range\": { \"date\": { \"gt\": \"2021-01-01\" }}}]}}}",
        "GET /_search { \"query\": { \"bool\": { \"must\": [{ \"match\": { \"severity\": \"high\" }}, { \"range\": { \"date\": { \"lt\": \"2021-01-01\" }}}]}}}",
        "GET /_search { \"aggs\": { \"status\": { \"terms\": { \"field\": \"status\" }}, \"aggs\": { \"severity\": { \"terms\": { \"field\": \"severity\" }}, \"aggs\": { \"age\": { \"terms\": { \"field\": \"age\" }}}}}}",
        "GET /_search { \"aggs\": { \"status\": { \"terms\": { \"field\": \"status\" }}, \"aggs\": { \"severity\": { \"terms\": { \"field\": \"severity\" }}, \"aggs\": { \"date\": { \"terms\": { \"field\": \"date\" }}}}}}",
        "GET /_search { \"aggs\": { \"severity\": { \"terms\": { \"field\": \"severity\" }}, \"aggs\": { \"age\": { \"terms\": { \"field\": \"age\" }}, \"aggs\": { \"date\": { \"terms\": { \"field\": \"date\" }}}}}}",
        "GET /_search { \"aggs\": { \"age\": { \"terms\": { \"field\": \"age\" }}, \"aggs\": { \"status\": { \"terms\": { \"field\": \"status\" }}, \"aggs\": { \"date\": { \"terms\": { \"field\": \"date\" }}}}}}",
        "GET /_search { \"query\": { \"wildcard\": { \"name\": \"A*\" }}}",
        "GET /_search { \"query\": { \"wildcard\": { \"name\": \"*Z\" }}}",
        "GET /_search { \"query\": { \"wildcard\": { \"name\": \"*BCD*\" }}}",
        "GET /_search { \"query\": { \"match\": { \"status\": \"pending\" }}}",
        "GET /_search { \"query\": { \"match\": { \"severity\": \"medium\" }}}",
        "GET /_search { \"query\": { \"match\": { \"priority\": \"high\" }}}",
        "GET /_search { \"query\": { \"match\": { \"priority\": \"low\" }}}",
        "GET /_search { \"query\": { \"match\": { \"priority\": \"medium\" }}}",
        "GET /_search { \"aggs\": { \"priority\": { \"terms\": { \"field\": \"priority\" }}, \"aggs\": { \"count\": { \"value_count\": { \"field\": \"_index\" }}}}}",
        "GET /_search { \"aggs\": { \"priority\": { \"terms\": { \"field\": \"priority\" }}, \"aggs\": { \"average\": { \"avg\": { \"field\": \"priority\" }}}}}",
        "GET /_search { \"aggs\": { \"priority\": { \"terms\": { \"field\": \"priority\" }}, \"aggs\": { \"sum\": { \"sum\": { \"field\": \"priority\" }}}}}",
        "GET /_search { \"aggs\": { \"priority\": { \"terms\": { \"field\": \"priority\" }}, \"aggs\": { \"min\": { \"min\": { \"field\": \"priority\" }}}}}",
        "GET /_search { \"aggs\": { \"priority\": { \"terms\": { \"field\": \"priority\" }}, \"aggs\": { \"max\": { \"max\": { \"field\": \"priority\" }}}}}"
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save DataFrame to CSV
df.to_csv('opensearch_finetuning_dataset.csv', index=False)

print("Dataset created and saved as 'opensearch_finetuning_dataset.csv'")


Dataset created and saved as 'opensearch_finetuning_dataset.csv'
